In [1]:
%reload_ext autoreload
%autoreload 2

import os
import xarray as xr
import numpy as np
import sys
import pandas as pd

sys.path.append("..")
import utils
import plot_utils

In [21]:
fold = f"../../results/MHW/satellite/"
baseline = 'fixed_baseline'

ds_fix = xr.open_mfdataset(f'{fold}fixed_baseline_20_year/*.nc')
ds_mov = xr.open_mfdataset(f'{fold}moving_baseline_20_year/*.nc')
ds_ts = xr.open_dataset("../../results/SST/satellite/mean_sst_satellite.nc")

clim = np.float64(ds_ts.sel(year = slice(1982,2001))['sst'].mean().values)
ans = np.zeros((20))
for i in range(20):
    ans[i] = np.float64(ds_ts['sst'].sel(year = 2002 + i).values) - clim

ans

array([0.44380569, 0.81829262, 0.2646637 , 0.25009727, 0.42185211,
       0.61092377, 0.65424156, 0.64835167, 0.74548721, 0.83198547,
       0.96738052, 0.70445824, 0.96484947, 1.04626083, 1.03606033,
       0.96164703, 1.20953369, 0.9218235 , 1.0315876 , 0.96917725])

In [25]:
years = np.arange(2002, 2022)
metrics = [
    "SST an.",
    "MHW (days)",
    "MHS (days)",
    "Mean anomaly",
    "Mean cumulative anomaly",
    "Mean duration (days)",
]
metrics_ = ["MHW", "MHS", "mean_anomaly", "cumulative_anomaly", "mean_duration"]
columns = pd.MultiIndex.from_product([metrics, ["Fix", "Mov"]])


df = pd.DataFrame(index=years, columns=columns)


for i, metric in enumerate(metrics):
    for year in years:
        if i!=0:
            df.loc[year, (metric, "Fix")] = np.float64(
                ds_fix[metrics_[i-1]].sel(time=year).mean(["lat", "lon"]).values
            )
            df.loc[year, (metric, "Mov")] = np.float64(
                ds_mov[metrics_[i-1]].sel(time=year).mean(["lat", "lon"]).values
            )
        else:
            df.loc[year, (metric, "Fix")] = ans[year-2002]

def custom_round(x):
    return round(x, 2)


df = df.applymap(custom_round)

df


SST an.     MHW (days)        MHS (days)         Mean anomaly        \
         Fix Mov        Fix    Mov        Fix     Mov          Fix   Mov   
2002    0.44 NaN      38.21  38.21      67.54   67.54         1.99  1.99   
2003    0.82 NaN      90.02  85.96     112.13  107.63         2.34  2.34   
2004    0.26 NaN      27.47  17.70      47.17   34.18         1.78  1.77   
2005    0.25 NaN      24.30  14.94      43.31   30.01         1.89  1.87   
2006    0.42 NaN      44.67  32.83      67.79   52.50         1.98  1.92   
2007    0.61 NaN      72.78  53.69     100.07   77.58         1.83  1.72   
2008    0.65 NaN      50.48  26.56      86.71   55.77         1.94  1.87   
2009    0.65 NaN      55.92  29.22      86.04   54.28         2.05  1.92   
2010    0.75 NaN      95.44  59.86     118.40   81.74         1.95  1.85   
2011    0.83 NaN     100.27  59.59     129.07   84.17         1.71  1.53   
2012    0.97 NaN     123.90  73.07     146.43   95.67         1.97  1.72   
2013    0.70 NaN      73.40  28.99     102.31   47.55         1.85  1.87   
2014    0.96 NaN     113.79  57.22     147.41   83.21         1.66  1.47   
2015    1.05 NaN     125.01  54.99     152.15   78.31         1.99  1.81   
2016    1.04 NaN     132.80  46.96     166.85   76.68         1.62  1.44   
2017    0.96 NaN     109.73  31.69     137.99   52.29         1.93  1.78   
2018    1.21 NaN     153.27  55.89     178.83   80.46         2.05  1.85   
2019    0.92 NaN     110.86  34.22     135.36   52.16         2.04  1.80   
2020    1.03 NaN     130.48  38.13     164.08   61.89         1.76  1.47   
2021    0.97 NaN     106.59  25.72     142.14   51.61         1.92  1.50   

     Mean cumulative anomaly         Mean duration (days)         
                         Fix     Mov                  Fix    Mov  
2002                   70.08   70.08                11.96  11.96  
2003                  214.81  206.84                27.12  27.22  
2004                   49.11   31.72                12.38  11.02  
2005                   46.20   28.50                11.25  10.27  
2006                   90.33   64.88                14.30  12.97  
2007                  134.27   92.69                16.00  14.67  
2008                   93.32   47.13                12.09   9.92  
2009                  112.97   56.69                13.75  10.67  
2010                  177.95  101.00                17.35  13.19  
2011                  172.72   94.28                20.62  17.91  
2012                  245.16  125.85                23.82  15.63  
2013                  134.61   53.96                14.50  12.16  
2014                  191.50   87.06                19.90  14.90  
2015                  245.58  100.33                21.23  15.63  
2016                  216.31   66.95                19.87  12.82  
2017                  211.80   56.43                20.13  10.08  
2018                  311.80  104.33                27.96  14.83  
2019                  221.08   59.10                23.35  13.51  
2020                  231.99   58.01                20.61  15.12  
2021                  200.93   37.95                18.74  10.89

In [8]:
df

MHW (days)        MHS (days)         Mean anomaly  ($^\circ$$\!$C)        \
            Fix    Mov        Fix     Mov                           Fix   Mov   
2002      38.21  38.21      67.54   67.54                          1.99  1.99   
2003      90.02  85.96     112.13  107.63                          2.34  2.34   
2004      27.47  17.70      47.17   34.18                          1.78  1.77   
2005      24.30  14.94      43.31   30.01                          1.89  1.87   
2006      44.67  32.83      67.79   52.50                          1.98  1.92   
2007      72.78  53.69     100.07   77.58                          1.83  1.72   
2008      50.48  26.56      86.71   55.77                          1.94  1.87   
2009      55.92  29.22      86.04   54.28                          2.05  1.92   
2010      95.44  59.86     118.40   81.74                          1.95  1.85   
2011     100.27  59.59     129.07   84.17                          1.71  1.53   
2012     123.90  73.07     146.43   95.67                          1.97  1.72   
2013      73.40  28.99     102.31   47.55                          1.85  1.87   
2014     113.79  57.22     147.41   83.21                          1.66  1.47   
2015     125.01  54.99     152.15   78.31                          1.99  1.81   
2016     132.80  46.96     166.85   76.68                          1.62  1.44   
2017     109.73  31.69     137.99   52.29                          1.93  1.78   
2018     153.27  55.89     178.83   80.46                          2.05  1.85   
2019     110.86  34.22     135.36   52.16                          2.04  1.80   
2020     130.48  38.13     164.08   61.89                          1.76  1.47   
2021     106.59  25.72     142.14   51.61                          1.92  1.50   

     Mean cumulative anomaly ($^\circ \!$C$\cdot$day)          \
                                                  Fix     Mov   
2002                                            70.08   70.08   
2003                                           214.81  206.84   
2004                                            49.11   31.72   
2005                                            46.20   28.50   
2006                                            90.33   64.88   
2007                                           134.27   92.69   
2008                                            93.32   47.13   
2009                                           112.97   56.69   
2010                                           177.95  101.00   
2011                                           172.72   94.28   
2012                                           245.16  125.85   
2013                                           134.61   53.96   
2014                                           191.50   87.06   
2015                                           245.58  100.33   
2016                                           216.31   66.95   
2017                                           211.80   56.43   
2018                                           311.80  104.33   
2019                                           221.08   59.10   
2020                                           231.99   58.01   
2021                                           200.93   37.95   

     Mean duration (days)         
                      Fix    Mov  
2002                11.96  11.96  
2003                27.12  27.22  
2004                12.38  11.02  
2005                11.25  10.27  
2006                14.30  12.97  
2007                16.00  14.67  
2008                12.09   9.92  
2009                13.75  10.67  
2010                17.35  13.19  
2011                20.62  17.91  
2012                23.82  15.63  
2013                14.50  12.16  
2014                19.90  14.90  
2015                21.23  15.63  
2016                19.87  12.82  
2017                20.13  10.08  
2018                27.96  14.83  
2019                23.35  13.51  
2020                20.61  15.12  
2021                18.74  10.89

In [10]:
latex_table = df.to_latex('../../tables/table_1.tex')
excel_table = df.to_excel('../../tables/table_1.xlsx')

/tmp/ipykernel_2263501/1358002921.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex('../../tables/table_1.tex')


In [34]:
# Earth radius
R = 6371
lat = 40*np.pi/180

lon = 0.0625*np.pi/180

lon_km = R*np.cos(lat)*lon
lon_km

5.323765978692914